In [1]:
!pip install -Uqq fastbook

In [2]:
from fastai import *
import numpy as np
import pandas as pd
from fastbook import *
import matplotlib.pyplot as plt
from fastai.vision.widgets import *

In [3]:
aircraft_types = ['F16 Fighting Falcon', 'F4 Phantom', 'F117 Nighthawk', 'F22 Raptor']
path = Path('Aircrafts')

if not path.exists():
    path.mkdir()
    for o in aircraft_types:
        print(f'Downloading pictures of {o}')
        dest = path/o
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o}')
        download_images(dest, urls=results)

In [ ]:
fns = get_image_files(path)
failed = verify_images(fns)

In [ ]:
failed.map(Path.unlink)

In [ ]:
acs = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct = 0.2, seed = 40),
    get_y = parent_label,
    item_tfms = Resize(128))

In [ ]:
dls = acs.dataloaders(path)

In [ ]:
acs = acs.new(
    item_tfms = RandomResizedCrop(225, min_scale = 0.5), 
    batch_tfms = aug_transforms())
dls = acs.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()


In [ ]:
interp.plot_top_losses(4, nrows = 1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [ ]:
learn.export()

In [4]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [5]:
learner_inf = load_learner(path/'export.pkl')

In [7]:
learner_inf.predict(path/'f22.jpg')

('F22 Raptor',
 TensorBase(2),
 TensorBase([1.2035e-08, 5.3973e-08, 1.0000e+00, 3.0286e-07]))

In [8]:
learner_inf.dls.vocab

['F117 Nighthawk', 'F16 Fighting Falcon', 'F22 Raptor', 'F4 Phantom']

In [20]:
btn_upload = widgets.FileUpload(multiple = False)
btn_upload

FileUpload(value={}, description='Upload')

In [8]:
pred, pred_idx, probs = learner_inf.predict(img)

NameError: name 'img' is not defined

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}, Probability: {probs[pred_idx]:.03f}'
lbl_pred

NameError: name 'pred' is not defined

In [14]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))

In [17]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learner_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [22]:
btn_run = widgets.Button(description = 'Classify')
btn_run.on_click(on_click_classify)

In [21]:
VBox([widgets.Label('Upload your aircraft!'),
     btn_upload, btn_run, out_pl, lbl_pred])